# **Solving PDEs with Gridap**

In this tutorial we will go through the fundamental theory of the Finite Element Method to solve PDEs. This theory expects little to no knowledge in FEM and is written as a roadmap which can be followed to solve most PDEs using Gridap. 

To apply the theory we will solve the Poisson equation. After each piece of theory, we apply the theory to the problem to demonstrate the workflow presented in the tutorial.

Let's begin!

## **What is a (Partial) Differential Equation P(DE)?**

To describe phenomena in the real world we need to understand the physics behind the phenomena. Physics can often be described in words, but most often we want to describe the physics using mathematical equations. For these kind of problems we always have two kind of variables: 

1. Independent variables: such as space ($\mathbf{x}$) and time ($t$).
2. Dependent variables: a variable that depends on the independent variables (for instance, the deformation of a beam along its span (thus space) and in time)

The relation between independent and dependent variables is given by a (partial) differential equation, along with initial and boundary conditions such that we can solve the (P)DE and find the ***dependent*** variables, which we initially do not know. Hence, a P(DE) only describes the relation between the dependent and independent variables, from which we can then *find* the unknown dependent variables.

In this theory we will mainly focus on **Partial Differential Equations** since these are regarded more difficult to solve using analytical approaches,in comparison to Ordinary Differential Equations. 

One of the most known PDEs is the ***Poisson Equation***. Hence, in this tutorial we will solve the ***Poisson Equation*** to demonstrate the purposed roadmap. Keep in mind that the purposed roadmap can be (in most cases) applicable for other PDEs.

# Solve Poisson equation on a 2D square

Basic steps for FEM:
1. Find the physical problem
2. Describe the mathematical model
    1. Governing equations (PDEs, boundary conditions, initial conditions)
    2. Constant parameters , source functions
    3. Define the unknown quantities to solve for
3. Create FE space to find the unknown quantities in.
    1. Create mesh
        1. Mesh density & mesh profile
        2. Type of FE spaces
        3. Conformity of the FE functions 
4. Writing the weak form of the problem.
5. Solving the problem.
6. Iterating for different mesh sizes until solution converges.

# **Step 1: Define physical problem**

The very first step for all problems is to define the physical problem. This step is for the largest part unconnected to the Finite Element Method but is essential in order to be able to continue. For this we need to understand some terminology and symbols which we often encounter in PDEs.
## **Scalar and vector valued variables**
For simplicity, we consider *two* type of variables: ***scalar valued variables*** and ***vector valued variables***.

### ***Scalar valued variables***
A ***scalar valued variable*** is a variable which, as the name says, is a scalar. Imagine we would like to know the deformation of a beam in space and time. Hence, we want to find a dependent variable, in this case the deformation of the beam denoted as $u$, as a function of the space and time (the independent variables). For a 1D beam we can describe the deformation as:
$$
    \text{deformation of the beam at } x_{0} \text{ and } t_{0} : u(x_{0},t_{0}) = \text{a scalar which describes the deformation at } x_{0} \text{ and } t_{0}
$$
Mathematically we denote this as:
$$
u : \mathbb{R}^d \times \mathbb{R} \rightarrow \mathbb{R}
$$
which means that the deformation can be explained be a function $u$ that has two inputs: the space of the beam, which has $d$ dimensions (for a 1D beam, $d$=1 etc.), and the second input is time (which is always "1D"). Note that for the deformation of a plate, the space would not be 1-dimensional anymore, since we then would consider $\mathbf{x} = (x,y)$. In that case our space is a *vector-valued variable*, but our dependent variable, $u$, remains a scalar-valued variable since the deformation of a plate is still a scalar along the space of the plate.

### ***Vector valued variables***
A ***vector valued variable*** is a variable which, as the name says, is a vector. Vector valued variables describe a quantity using a vector, and not as a scalar. One example is the velocity of a fluid. Imagine we want to describe the velocity of a fluid in a 2D space. The velocity at any point in space and time can be described by a vector with two components (one for each spatial dimension). Mathematically, we denote this as:

$$
\mathbf{v} : \mathbb{R}^d \times \mathbb{R} \rightarrow \mathbb{R}^d
$$

which means that the velocity can be explained by a function $\mathbf{v}$ that has two inputs: the space of the fluid, which has $d$ dimensions (for a 2D space, $d$=2), and the second input is time. The output of this function is a vector with $d$ components, each representing the velocity in one of the spatial dimensions. For example, in a 2D space, the velocity at a point $(x_0, y_0)$ and time $t_0$ can be written as:

$$
\mathbf{v}((x_0, y_0), t_0) = \begin{pmatrix} v_x((x_0, y_0), t_0) \\ v_y((x_0, y_0), t_0) \end{pmatrix}
$$

where $v_x$ and $v_y$ are the velocity components in the $x$ and $y$ directions, respectively.

## **Symbols**

The Finite Element Method relies on a strong understanding of mathematics and is often formulated using mathematical symbols which *set* the "rules" and describe the relation between the dependent and dependent variables. 

### **Operators**

In the context of PDEs and FEM, operators are mathematical tools that help us describe the relationships between variables. Here are some common operators:

1. **Gradient ($\nabla$)**: The gradient operator takes a scalar field and produces a vector field. It represents the rate and direction of change in the scalar field. For a function $u(x, y)$, the gradient is:
    $$
    \nabla u = \begin{pmatrix} \frac{\partial u}{\partial x} \\ \frac{\partial u}{\partial y} \end{pmatrix}
    $$

2. **Divergence ($\nabla \cdot$)**: The divergence operator takes a vector field and produces a scalar field. It measures the magnitude of a source or sink at a given point. For a vector field $\mathbf{v} = (v_x, v_y)$, the divergence is:
    $$
    \nabla \cdot \mathbf{v} = \frac{\partial v_x}{\partial x} + \frac{\partial v_y}{\partial y}
    $$

3. **Laplacian ($\Delta$)**: The Laplacian operator is the divergence of the gradient of a scalar field. It is often used in diffusion problems. For a function $u(x, y)$, the Laplacian is:
    $$
    \Delta u = \nabla \cdot (\nabla u) = \frac{\partial^2 u}{\partial x^2} + \frac{\partial^2 u}{\partial y^2}
    $$

### **Boundary Conditions**
Boundary and initial conditions are essential for solving Partial Differential Equations (PDEs) because they provide the necessary information to obtain a ***unique*** solution. 

Boundary conditions are, as the name says, used to assign the value of the unknown dependent variable at the boundaries of the considered domain. Boundary conditions are essential to solve the problem in *space*.

Boundary conditions specify the behavior of the solution at the boundaries of the domain. There are typically two types of boundary conditions:
1. **Dirichlet Boundary Condition**: Specifies the value of the solution at the boundary. For example, if we are solving for temperature distribution, a Dirichlet boundary condition might specify the temperature at the boundary.
    $$
    u = g \quad \text{on } \Gamma_D
    $$

2. **Neumann Boundary Condition**: Specifies the value of the derivative (flux) of the solution normal to the boundary. For example, in heat transfer problems, a Neumann boundary condition might specify the heat flux at the boundary.
    $$
    \nabla u \cdot \mathbf{n} \equiv \frac{\partial u}{\partial n} =  h \quad \text{on } \Gamma_N
    $$

Often it can be difficult to understand the definition of a "boundary", especially for beginners. However, a **boundary** of a **domain** "lives" in the *(n-1)*-dimension of a domain that is defined in *n*-dimensions. 

That is, let $\Omega \subset \mathbb{R}^n$ be an open domain where the governing PDE is defined. The $\textbf{boundary}$ of the domain, denoted as $\partial \Omega$, is the set of points that separate the interior of $\Omega$ from its exterior.

Mathematically:
$$
\partial \Omega = \overline{\Omega} \setminus \Omega,
$$
where:
1. $\overline{\Omega}$: The $\textbf{closure}$ of $\Omega$, which includes all points in $\Omega$ and all its boundary points.
2. $\Omega$: The $\textbf{interior}$ of the domain, excluding the boundary. 
3. $\setminus$: Denotes the set difference. 
    
In simpler terms, $\partial \Omega$ consists of all points $\mathbf{x} \in \mathbb{R}^n$ that act like a barrier between the domain $\Omega$ and its complement $\mathbb{R}^n \setminus \Omega$ (the "outside world").

![Boundary Illustration](./images/boundaries.png)

### **Determining the Number of Boundary Conditions**
Determining the number of boundary conditions can be challenging, especially if the boundary can be split in multiple separate components. There are two important points we should consider:
1. The minimum number of boundary conditions needed is equal to the order of the PDE (that is the order of the highest derivative in the governing equation).
2. The behaviour around ***all*** boundaries should be defined. 
3. The boundary conditions should ***not*** contradict each other (can be the case when there are *too* many boundary conditions defined, without ensuring the physical validity of the BCs).

### **Initial Conditions**
Initial conditions specify the state of the system at the beginning of the time domain. For time-dependent problems, initial conditions are necessary to start the simulation. For example, in a heat conduction problem, the initial temperature distribution must be specified.

Initial conditions are easy to define. The rule is that for a *(n-th)*-order derivative in time present in the governing equation, we need *n-1* initial conditions that are of lower order-derivative than the one present in the governing equation.

Formally, the problem to solve is: find the scalar field $u(\mathbf{x}, t)$ such that


\begin{cases} 
-\Delta u = f & \text{in } \Omega, \\
u = g & \text{on } \Gamma_D, \\
\nabla u \cdot \mathbf{n} = h & \text{on } \Gamma_N,
\end{cases}

In [1]:
f(x) = 1.0
g(x) = 2.0
h(x) = 3.0

h (generic function with 1 method)

In [5]:
using Gridap
# using Plots
using GridapMakie
using CairoMakie
using GLMakie

domain = (0.0, 1.0, 0.0, 1.0)  # Define a square domain
partition = (10,10)  # Mesh resolution
model = CartesianDiscreteModel(domain, partition)

CartesianDiscreteModel()

In [3]:
labels = get_face_labeling(model)
# 0-faces: vertices: points
# 1-faces: edges: lines
# 2-faces: faces: surfaces 
# 3-faces: cells: volumes 

Gridap.Geometry.FaceLabeling:
 0-faces: 121
 1-faces: 220
 2-faces: 100
 tags: 10
 entities: 9

Since we are considering the Poisson equation in 2D, we find that we have a total of 9 entities:

* We have 4 vertices.
* We have 4 edges
* We have 1 face

All these *entities* together make 9. Now the following step is to gives names to these entities, such that later when we solve the problem we can easily assign the boundary conditions. 

So from the above cel code we have the numbered labels of the model, and now we want to change these numbered labels to texted names. 

In [38]:
add_tag_from_tags!(labels, "Dirichlet", [5,7,8])
add_tag_from_tags!(labels, "Neumann", [6])

12-element Vector{String}:
 "tag_1"
 "tag_2"
 "tag_3"
 "tag_4"
 "tag_5"
 "tag_6"
 "tag_7"
 "tag_8"
 "interior"
 "boundary"
 "Dirichlet"
 "Neumann"

In [39]:
Ω = Triangulation(model)
Γ_D = Boundary(Ω,tags="Dirichlet")
Γ_N = Boundary(Ω,tags="Neumann")

CompositeTriangulation()

# Finite Element spaces

In [46]:
order = 1
reffe = ReferenceFE(lagrangian, Float64, order)
V0 = TestFESpace(Ω, reffe; conformity=:H1, dirichlet_tags="Dirichlet")
U0 = TrialFESpace(V0, g)

TrialFESpace()

In [49]:
degree = 2
dΩ = Measure(Ω,degree)
dΓ_D  = Measure(Γ_D,degree)
dΓ_N = Measure(Γ_N,degree)

GenericMeasure()

In [51]:
a(u,v) = ∫( ∇(v)⋅∇(u) )*dΩ
b(v) = ∫( v*f )*dΩ + ∫( v*h )*dΓ_N

b (generic function with 1 method)

In [52]:
op = AffineFEOperator(a,b,U0,V0)

AffineFEOperator()

In [53]:
ls = LUSolver()
solver = LinearFESolver(ls)

uh = solve(solver,op)

SingleFieldFEFunction():
 num_cells: 100
 DomainStyle: ReferenceDomain()
 Triangulation: BodyFittedTriangulation()
 Triangulation id: 15381795345852993462

In [ ]:
# writevtk(Ω,"results",cellfields=["uh"=>uh])